In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import load_model
import cv2, os
from os import listdir
from os.path import isfile, join
import scipy
from skimage import transform
import preprocess
from nvi_utils import random_shadow, random_brightness

IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS = 66, 200, 3
INPUT_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)

#from ironcar import Ironcar

Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers import Lambda, Conv2D, Dropout, Dense, Flatten

def build_model():                                                              
    """                                                                         
    Modified NVIDIA model                                                       
    """                                                                         
    model = Sequential()                                                        
    model.add(Lambda(lambda x: x/127.5-1.0, input_shape=INPUT_SHAPE))           
    model.add(Conv2D(24, 5, 5, activation='elu', subsample=(2, 2)))             
    model.add(Conv2D(36, 5, 5, activation='elu', subsample=(2, 2)))             
    model.add(Conv2D(48, 5, 5, activation='elu', subsample=(2, 2)))             
    model.add(Conv2D(64, 3, 3, activation='elu'))                               
    model.add(Conv2D(64, 3, 3, activation='elu'))                               
    model.add(Dropout(0.5))                                                     
    model.add(Flatten())                                                        
    model.add(Dense(100, activation='elu'))                                     
    model.add(Dense(50, activation='elu'))                                      
    model.add(Dense(10, activation='elu'))
    model.add(Dense(1))                                                      

    return model 

In [ ]:
model_name = "./models/model-0,5YUV.h5"
model = build_model()
model.load_weights(model_name)

img = cv2.imread("../AutoCar/road_simulator/src/dataset_simulator/frame_1402_gas_0.5_dir_0.20146169281252368.jpg")
img = transform.resize(img, (IMAGE_HEIGHT, IMAGE_WIDTH))
# replace random_shadow by random_brightness to test brightness
shadow = random_shadow(img)

import matplotlib.pyplot as plt

# RGB TO BGR for plt
img_disp = img[:, :, ::-1]
shadow_disp = shadow[:, :, ::-1]
plt.imshow(img_disp)
plt.figure()
plt.imshow(shadow_disp)

model.predict(np.array([preprocess.rgb2yuv(img), preprocess.rgb2yuv(shadow)]))

In [3]:
model_name = "./models/model-0,0YUV.h5"
model = build_model()
model.load_weights(model_name)

funs = [(lambda x: x), random_shadow, random_brightness]
root = "../AutoCar/road_simulator/src/dataset_simulator/"
res = []

for fun, i in zip(funs, range(len(funs))):
    files = [root + f for f in listdir(root) if isfile(join(root, f))]
    imgs = [ img for img in map(cv2.imread, files) ]
    imgs = [ img for img in map(lambda x: transform.resize(x, (IMAGE_HEIGHT, IMAGE_WIDTH)), imgs)]
    imgs = [ img for img in map(fun, imgs)]
    imgs = [ img for img in map(preprocess.rgb2yuv, imgs)]
    
    prediction = model.predict(np.array(imgs))
    res.append(prediction)
    
print("shadow")
diff = abs(res[0] - res[1])
mean = np.mean(diff)
print(mean)

df = pd.DataFrame(diff, index=['Row'+str(i) for i in range(len(diff))])
print(df.describe())

print("light")
diff2 = abs(res[0] - res[2])
mean2 = np.mean(diff2)
print(mean2)

df2 = pd.DataFrame(diff2, index=['Row'+str(i) for i in range(len(diff2))])
print(df2.describe())

"""
for f in files:
    file_path = root + f
    img = cv2.imread(file_path)
    img = np.array([img])
    prediction.append(model.predict(img))


root = "./blur/"
files = [f for f in listdir(root) if isfile(join(root, f))]
blur_pred = []

for f in files:
    file_path = root + f
    img = mpimg.imread(file_path)
    img = np.array([img])
    blur_pred.append(model.predict(img))

print(np.array(prediction))
#print(np.array(blur_pred))
"""

C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="elu", strides=(2, 2))`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="elu", strides=(2, 2))`
  # This is added back by InteractiveShellApp.init_path()
C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="elu", strides=(2, 2))`
  if sys.path[0] == '':
C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="elu")`
  del sys.path[0]
C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3

error: C:\ci\opencv_1512688052760\work\modules\imgproc\src\color.cpp:11010: error: (-215) depth == 0 || depth == 2 || depth == 5 in function cv::cvtColor


In [ ]:
print(df.idxmax())
print(files[4302])

print(df2.idxmax())
print(files[4391])